In [16]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

In [2]:
Map = emap.Map(center=[18,-66], zoom=9)
Map.add_basemap('SATELLITE') # Add Google Map
Map

Map(center=[18, -66], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [4]:
# setting Puerto Rico boundary
PR_polygon = ee.Geometry.Polygon([[[-66.97540283203125, 18.04403274297545],\
                                 [-65.93170166015625, 18.04403274297545],\
                                 [-65.93170166015625, 18.41447273166262],\
                                 [-66.97540283203125, 18.41447273166262],\
                                 [-66.97540283203125, 18.04403274297545]]])


In [5]:
# cloud mask function
def maskS2clouds(image):
#     bits 10 and 11 are cloud and cirrus 
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
#     both needs to be set to zero - clear condition
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
    qa.bitwiseAnd(cirrusBitMask).eq(0))
#     return the masked and scaled data without the QA bands
    return image.updateMask(mask).divide(10000)\
        .select("B.*")\
        .copyProperties(image, ['system:time_start'])

In [6]:
# get level 2 sen2 image 

Start = ee.Date('2017-03-28')
End = ee.Date('2017-12-31')
img = ee.ImageCollection('COPERNICUS/S2')\
    .filterDate(Start, End)\
    .filterBounds(PR_polygon)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
    .map(maskS2clouds)
composite = img.median()


In [26]:
Map.addLayer(composite, {'bands':['B8', 'B4', 'B3'], 'min': 0, 'max':0.3}, 'False IR')

In [12]:
# input farm data
file_path = 'PR_farms.json'
with open(file_path) as f:
    json_data = json.load(f)

In [17]:
json_layer = GeoJSON(data = json_data, name = 'Small Holder Farmers JSON', hover_styler ={'red'})
Map.add_layer(json_layer)

In [22]:
# converted geojson data into ee layer
ee_data = geojson_to_ee(json_data)
Map.addLayer(ee_data, {}, "Small Holder Farms EE")

In [25]:
buffered = ee_data.map(lambda f: f.buffer(200))
unioned = buffered.union()
Map.addLayer(unioned, {'color': '800080'}, 'Small Holder Farms Buffer')